# Profiling Applications
---

The purpose of this notebook is to teach the basic concept of profiling deep neural networks with multi-GPU. You will learn how to use the NVIDIA Tools Extension (NVTX) tag to annotate your code and the command to profile it and analyze it using the NVIDIA® Nsight™ Systems GUI. Please note that your result might vary based on the GPU architecture used for the profile. All results are based on DGX A100.

For learning purposes, we will consider using the DDP code from the previous notebook (distributed data parallelism) with multi-GPU.
To start, use the NVIDIA® Nsight™ Systems command line interface (CLI) to profile our application. The rule of thumb is to limit the profiling to the second or third epoch. The first epoch is meant to “warm up” the GPU, and the second and subsequent epochs would have the same workload. We will be using the APIs `cudaProfilerStart` to set the profiling start point and `cudaProfilerStop` to mark the stop region within our application as shown below. For this section, our profiling will be limited to just the `second epoch` throughout the optimization workload.

```python
...
scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
for epoch in range(1, args.epochs + 1):
        # Start profiling from 2nd epoch
        if epoch == 2:
            torch.cuda.cudart().cudaProfilerStart()
        train(args, model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)
        scheduler.step()
        # Stop profiling at the end of 2nd epoch
        if epoch == 2:
            torch.cuda.cudart().cudaProfilerStop()
...
```

Below is the command to run when profiling an application.

```python

!nsys profile --trace cuda,osrt \
--capture-range cudaProfilerApi \
--output ../reports/baseline \
--force-overwrite true \
python3 <application-name>.py

```


Below is a brief glossary of the profiling command line used.

- `nsys`: command to call Nsight Systems CLI 
- `profile`: command to instruct profiling
- `--trace`: flag that represent tracing of APIs 
- `cuda,osrt`: APIs to be traced 
- `--capture-range`:flag that indicate start and stop profiling range using `cudaProfilerStart()` and `CudaProfilerStop()` APIs. 
- `--output`: flag that indicate the output of the profile  
- `../reports/baseline`: directory to save the profile output as `basline.qdrep` or `baseline.nsys-rep` 

- `--force-overwrite`: flag that denotes overwrite existing output file. It is usually set as `true` or `false`

- `python3`: command to run the DNN code in linux/ubuntu OS
- `<application-name>.py`: path to the DNN python code

A detailed user guide on Nsight Systems CLI profiling commands is presented [here](https://docs.nvidia.com/drive/drive-os-5.2.3.0L/nsight-systems/pdf/UserGuide.pdf). 

Let's run a simple [DDP program](../source_code/ddp_baseline.py) with 4 GPUs and time the execution duration. The program uses the resnet18 model and the CIFAR10 dataset.


In [ ]:
!cd ../source_code && srun -p gpu -N 1 --gres=gpu:4 torchrun  --nproc_per_node=4 --nnodes=1 --standalone --master_addr="localhost" --master_port=1234 ddp_baseline.py

**Likely output on DGX A100:**

```python
...
Local Rank: 2, Epoch: 23, Training ...
Local Rank: 1, Epoch: 23, Training ...
Local Rank: 3, Epoch: 23, Training ...
Local Rank: 0, Epoch: 23, Training ...
Local Rank: 2, Epoch: 24, Training ...
Local Rank: 1, Epoch: 24, Training ...
Local Rank: 3, Epoch: 24, Training ...
Local Rank: 0, Epoch: 24, Training ...
Total elapsed time: 100.95 seconds
Total elapsed time: 101.43 seconds
Total elapsed time: 101.46 seconds
Total elapsed time: 101.50 seconds

```

It takes 101.50 seconds to execute the 25 epochs in the training program.

### Basic Performance Issues to Address 

There are 3 three basic performance issues that are common to DNN applications, and these include:
- Data loading
- Data transfer (H to D with `cudaMemcpyAsync`)
- Absent of Tensor Core usage

We will demonstrate how to identify these basic issues by profiling a DDP application. But before we proceed, we need to learn about the [NVIDIA Tools Extension](https://nvtx.readthedocs.io/en/latest/index.html) (NVTX) annotation library to annotate our application. The annotation helps to easily identify which part of our code has bottlenecks and hotspots to address.  The NVTX library provides functions for annotating events, code ranges, and resources in your application. The NVTX APIs offer additional information for NVIDIA’s tools while incurring almost no overhead when the tool is not attached to the application. A sample code to add NVTX annotations looks like the following:   

```python
    from torch.cuda import nvtx
    
    ....

    nvtx.range_push("annotation string")

     #lines of code to annotate

    nvtx.range_pop(); 
    
    .....

```

Let's examine our simple [DDP application](../source_code/ddp-baseline_nvtx.py). The inclusion of `nvtx` in the `--trace` flag enables the trace of the code where `nvtx` annotation is specified. Open the report with NVTX annotations `(baseline_nvtx.nsys-rep file)` in the GUI and zoom in to find the repeating pattern of 'GPU idle' followed by `a burst of kernel and memory operations`. 

In [ ]:
!cd ../source_code && module load nsight-systems/2023.3 && srun -p gpu -N 1 --gres=gpu:4 nsys profile --trace cuda,osrt,nvtx \
--capture-range cudaProfilerApi \
--gpu-metrics-device=all \
--output ../reports/baseline_nvtx \
--force-overwrite true \
torchrun  --nproc_per_node=4 --nnodes=1 --standalone --master_addr="localhost" --master_port=1234 ddp-baseline_nvtx.py

When the profiling is done, we will inspect the report in the Nsight Systems' graphical user interface (GUI). Download the [baseline_nvtx.nsys-rep](../reports/baseline.nsys-rep) file at `../reports/baseline_nvtx.nsys-rep` and view in the NVIDIA Nsight Systems GUI. 

**Data loading Inspection**

The NVTX annotation in the report below shows the time to load data (data loading).  As a result, all the GPUs have significant idle time. This GPU idle time repeats at intervals as data loading takes place after each forward and backward pass. 

<center><img src="images/baseline_dataloading.png" width="800px" height="800px" alt-text="workflow"/></center>
</br>

To reduce the data loading and GPU idle time, we should increase the number of data loader workers in our DDP program as follows:

```python

# Increase the number of workers from 1 to 2 
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, sampler=train_sampler, num_workers=2)

test_loader = DataLoader(dataset=test_set, batch_size=128, shuffle=False, num_workers=2)

```
*Please note that in real-life applications with multi-GPU, the effect of more workers may not be visible due to some trade-off that happens as a result of other optimization steps.* 

**Data transfer Inspection**

To analyze the data transfers between host (H) and device (D) in the report, expand the NVIDIA CUDA® HW row of each GPU by clicking on the tiny triangle in front of it. Select the Memory row and right-click to choose the `Show in Events View` option, as shown below.

<center><img src="images/baseline-show-event-view.png" width="800px" height="800px" alt-text="workflow"/></center>
<br/><br/>

This populates the Events View window with the memory operations listed in chronological order. Click on the Duration column header to sort the table in the Events View by duration so that the longest memory operation appears first. Right-click on the first entry in the table and select `Zoom to Select on Timeline,` as illustrated below.


<center><img src="images/baseline-pageable.png" width="800px" height="800px" alt-text="workflow"/></center>
<br/>


This zooms into the event on the timeline, and the teal highlights help you find the CUDA API call, `cudaMemcpyAsync,` that initiated the memory operation on the GPU (see the image below).  You notice the following from the timeline:
- All Host-to-Device (HtoD) memcopies are using pageable memory, which is slower and causes the `cudaMemcpyAsync` API call on the CPU thread to block until the operation completes on the GPU.
- The longest memcpy operation takes ~603,000 $\mu$s on `GPU1` and 431,262 $\mu$s on `GPU2.`
- The CUDA API call `(cudaMemcpyAsync)` corresponding to the longest memcpy operations on `GPU1` is more than 14,876 $\mu$s, while on `GPU2` it is almost 503,100 $\mu$s long.

<center><img src="images/baseline-cudamemcpyAsync.png" width="1000px" height="1000px" alt-text="workflow"/></center>
<br/>

To reduce the time taken by the CUDA API call `(cudaMemcpyAsync),` we change from `pageable` memory to `pinned` memory, so host data are directly allocated to pinned memory without using Pageable memory.


```python

# Add pin_memory=True to the DataLoader
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, sampler=train_sampler, num_workers=2, pin_memory=True)

test_loader = DataLoader(dataset=test_set, batch_size=128, shuffle=False, num_workers=2, pin_memory=True) #optional

```

**Tensor Core Inspection**

Tensor cores are specialized processing units designed to accelerate tensor/matrix multiplication. They enable mixed-precision computing, dynamically adapting calculations to accelerate throughput while preserving accuracy. Our application runs on the NVIDIA® DGX™ A100 Ampere and Hopper H100 GPU architectures. The use of `--gpu-metrics-device=all` helps capture performance metrics samplings like SM instruction that include active tensor and NVLink(Rx & Tx) Bandwidth.

<center><img src="images/architecture_tensor_cores.jpg" width="700px" height="700px" alt-text="workflow"/></center>
<center><a href="https://www.nvidia.com/en-us/data-center/tensor-cores/" > View source </a></center>

By default, these Architectures enable tensor cores (FP32) for DNN applications. To check for tensor core usage in the profile report, click on any of the GPUs (for example, `GPU (000:47:00.0-NVIDIA A100-SXM4-80GB)`) and dropdown `GPU Metrics [10kHz].` Click on `SM Instructions` to see `Tensor Active.` When you hover your in the timeline, the popup box will not display the type of the tensor core. We will discuss where to you can find the tensor core used in the following notebook. 

<center><img src="images/baseline-tensor-core.png" width="850px" height="850px" alt-text="workflow"/></center>
<br/>

For learning purposes, we will add automatic mixed precision AMP (FP16) to our program using the approach learned in the [previous notebook](other-topics.ipynb) (adding `torch.autocast` and `GradScaler`).

```python
...

fp16_scaler = torch.amp.GradScaler("cuda")
...
 with torch.amp.autocast(device_type='cuda', dtype=torch.float16, enabled=True):
                    nvtx.range_push("Copy to device")
                    inputs, labels = data[0].to(device), data[1].to(device)
                    nvtx.range_pop() # Copy to device
                    
                    nvtx.range_push("Forward pass")
                    optimizer.zero_grad()
                    outputs = ddp_model(inputs)
                    loss = criterion(outputs, labels)
                    nvtx.range_pop() # Forward pass
                
                nvtx.range_push("Backward pass")
                fp16_scaler.scale(loss).backward()    
                fp16_scaler.step(optimizer)
                fp16_scaler.update()
                nvtx.range_pop() # Backward pass
...

```

### Profile to Verify the Optimization

The next step is to profile again and verify if the code changes address the bottlenecks. Please run the command in the cell below.

In [ ]:
!cd ../source_code && module load nsight-systems/2023.3 && srun -p gpu -N 1 --gres=gpu:4 nsys profile --trace cuda,osrt,nvtx \
--capture-range cudaProfilerApi \
--gpu-metrics-device=all \
--output ../reports/firstOptim \
--force-overwrite true \
torchrun  --nproc_per_node=4 --nnodes=1 --standalone --master_addr="localhost" --master_port=1234 ddp_optimize.py

Download the report `firstOptim.nsys-rep` and open it in the Nsight Systems GUI. Zoom in on the data loading and data transfer (HtoD) on the event view timeline. You will notice a reduction in the time spent on both tasks. The report below shows that the time taken to load data on both GPU0 and GPU2 has reduced by more than 50%. 

<center><img src="images/opt-dataloading.png" width="850px" height="850px" alt-text="workflow"/></center>
<br/>





Before switching to pinned memory, the longest process (HtoD) on GPU1 (`process id: #7051`) took `~603,000` microseconds, and `431,262` microseconds on GPU2 (`process id:#4051`). After the optimization steps, the longest processes on both devices (`process id:6408`) are now noticeably lesser. 
<center><img src="images/opt-GPU1-pinned.png" width="850px" height="850px" alt-text="workflow"/></center>
<br/>

It is important to note that a process ID can be assigned to another process at the next application run. This was the case with process IDs `#7051 (HtoD)` and `#4051 (HtoD),` as they were assigned to a new process, `DtoD` on both devices (*as indicated in the report screenshot with green frames*)


<center><img src="images/opt-GPU2-pinned.png" width="850px" height="850px" alt-text="workflow"/></center>
<br/>




### Compare the Performance Before and After the Optimizations

Now that we have addressed some fundamental performance problems let's time the [DDP application](../source_code/ddp_run_optimize.py). Please run the command in the cell below. 

In [ ]:
!cd ../source_code && srun -p gpu -N 1 --gres=gpu:4 torchrun  --nproc_per_node=4 --nnodes=1 --standalone --master_addr="localhost" --master_port=1234 ddp_run_optimize.py

**Likely Output on DGX A100:**

```python
...
Local Rank: 2, Epoch: 23, Training ...
Local Rank: 1, Epoch: 23, Training ...
Local Rank: 3, Epoch: 23, Training ...
Local Rank: 0, Epoch: 23, Training ...
Local Rank: 2, Epoch: 24, Training ...
Local Rank: 0, Epoch: 24, Training ...
Local Rank: 3, Epoch: 24, Training ...
Local Rank: 1, Epoch: 24, Training ...
Total elapsed time: 61.28 seconds
Total elapsed time: 57.58 seconds
Total elapsed time: 62.31 seconds
Total elapsed time: 62.68 seconds
```

Comparing the time taken to run our baseline [DDP program code](../source_code/ddp_baseline.py) with the basic level [optimization DPP code](../source_code/ddp_run_optimize.py), we see that the overall time taken has reduced, as shown in the table below.

|DDP code|Time|Speedup|
|--|--|--|
|Baseline|101s| - |
|Basic optimized|62s|~1.63s |

Now that we have learned basic multi-GPU profiling, let's continue to the next notebook to explore more features of the NVIDIA Nsight System and reveal further insight about our DDP program. Please click the [Next Link](nsys-trace.ipynb).




---

## Links and Resources


[NVIDIA Nsight Systems](https://developer.nvidia.com/nsight-systems)


**NOTE**: To be able to see the profiler output, please download the latest version of NVIDIA Nsight Systems from [here](https://developer.nvidia.com/nsight-systems/get-started).


You can also get resources from [Open Hackathons technical resource page](https://www.openhackathons.org/s/technical-resources)


--- 

## Licensing 

Copyright © 2025 OpenACC-Standard.org. This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0). These materials may include references to hardware and software developed by other entities; all applicable licensing and copyrights apply.